# Adding PowerShell Code to SQL Notebooks

SQL Notebooks are in constant eveolution but as at June 2019, this is how I enable clients to run PowerShell Code in SQL Notebooks in the Markdown blocks

## Use the commands

In Azure Data Studio if you click on the cog in the bottom left and choose Keyboard shortcuts, you can see and change the Keyboard Shortcuts. This enables you to see the command that is called when you use a keyboard shortcut. For Example, if you look at

![image](https://user-images.githubusercontent.com/6729780/59566321-84233d80-9056-11e9-9643-e9e15e85a2f0.png)

You can see that the command that is called is `workbench.action.terminal.focus`

It turns out that you can call this as a link in a Markdown document using HTML with `<a href="">` and adding `command:` prior to the command. When the link is clicked the command will run.

So a link like this `<a href="command:workbench.action.terminal.focus">Click this link to open the terminal</a>` will open the terminal

<a href="command:workbench.action.terminal.focus">Click this link to open the terminal</a> 



I expect you have clicked this and it has not worked. There is an extra step that is required to enable this functionality.

## The Notebook needs to be Trusted - With a BIG WARNING

When you create a new notebook and save it you will see

![image](https://user-images.githubusercontent.com/6729780/59566360-365b0500-9057-11e9-87fb-1f8cbbb6e9e2.png)

If you click Not Trusted the Notebook will be trusted and allowed to run commands from the markdown blocks. 

**This is not something to be done lightly**. I was telling some of my security red team friends at the European PowerShell Conference about this functionality and they were very very interested in a product which has a link which if clicked by a user can execute code. So please, ***be very careful indeed with Notebooks from untrusted sources***. You can also use the steps below to examine the code

So with that big warning covered, lets add some PowerShell to a Notebook.


## Send a command sequence

In the [Visual Studio Code documentation](https://code.visualstudio.com/docs/editor/integrated-terminal) it states that 

![image](https://user-images.githubusercontent.com/6729780/59566689-5260a580-905b-11e9-8369-c7e4b9a0e229.png)

So we can use `workbench.action.terminal.sendSequence` to send code to the terminal. 

However, we still need to craft the command so that it will work as a link. 
It needs to be converted into a URL. 

I use this website [https://www.url-encode-decode.com/](https://www.url-encode-decode.com/) to do this.

This is **how you can check the code in other peoples notebook, use the decode capability.**

![image](https://user-images.githubusercontent.com/6729780/59567164-e5044300-9061-11e9-802b-7b28c3aee345.png)

Encoding `Set-Location C:\dbachecks` gives `Set-Location+C%3A%5Cdbachecks`

Unfortunately, that does not complete the actions required

The + needs to be replaced with a space or `%20`

You also need to double the `\` and replace the `%3A` with a `:`

The `"` needs to be replaced with `\u022`, the `'` with `\u027`, the curly braces won't work unless you remove the `%0D%0A`





So now you can write your PowerShell and encode it and add `\u000D` at the end of the code to pass an enter to run the code.

and then place all of that into a link like this

`<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 PLACE THE ENCODED CODE HERE %22%7D">Link Text</a>`

This means that to add the code

    Set-Location C:\dbachecks
    Get-ChildItem

You would end up with a link like this


`<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 Set-Location C:%5C%5Cdbachecks \u000D Get-ChildItem \u000D %22%7D">Set Location and list files</a>`

Who wants to remember all that everytime ? So I created a PowerShell function!


        function Convert-ADSPowerShellForMarkdown {
    <#
    .SYNOPSIS
    Converts PowerShell code into valid Markdown URI Link text
    
    .DESCRIPTION
    Converts PowerShell code into valid Markdown URI Link Text
    
    .PARAMETER inputstring
    The endoded URL from the website
    .PARAMETER LinkText
    The text to show for the link
    
    .PARAMETER ToClipBoard
    Will not output to screen but instead will set the clipboard
    
    .EXAMPLE
    Convert-ADSPowerShellForMarkdown -inputstring "Set-Location C:\temp" -LinkText "CLick this link to set the location" -ToClipboard
    Converts the PowerShell code Set-Location C:\temp so that it works with MarkDown, creates a link and sets it to the clipboard
        
        .NOTES
        June 2019 - Rob Sewell @SQLDbaWithBeard
        SQLDBAWithABeard.Com
        #>
        
        Param(
            [Parameter(Mandatory = $true)]
            [string]$inputstring,
            [string]$linktext = " LINK TEXT HERE ",
            [switch]$ToClipBoard
        )
        clear
        $linkage = $inputstring.Replace('+',' ').Replace('%3A',':').Replace('%5C','%5C%5C').Replace('%22','\u0022').Replace('%27','\u0027').Replace('%0D%0A','')
        
        $outputstring = @"
    <a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 $linkage \u000D %22%7D">$linktext</a>
    "@
        if($ToClipBoard){
            if (-not ($IsLinux -or $IsMacOS) ) {
            $outputstring | Set-Clipboard
            }
            else{
                Write-Warning "Set-Clipboard - Doesnt work on Linux - Outputting to screen" 
                $outputstring
            }
        }else {
            $outputstring
        }
    }

which I will keep updated [on GitHub](https://github.com/SQLDBAWithABeard/Functions/blob/master/Convert-ADSPowerShellForMarkdown.ps1)


# Run some PowerShell

So to run a bit of PowerShell we can do this

<a href="command:workbench.action.terminal.focus">Open a new terminal</a>

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 pwsh \u000D %22%7D">Start pwsh</a>

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 if(%24IsLinux)%7b\u000A    if(-not (Get-PSDrive -Name Git -ErrorAction SilentlyContinue))%7b\u000A        New-PSDrive -Name Git -PSProvider FileSystem -Root %2fhome%2frob%2fDocuments%2fgit\u000A    %7d\u000A    Set-Location \u0027%2fhome%2frob%2fDocuments%2fgit%2fPresentations%2f2019%2fSQL Saturday Oslo\u0027\u000A%7delse%7b\u000A    if(-not (Get-PSDrive -Name Git -ErrorAction SilentlyContinue))%7b\u000A        New-PSDrive -Name Git -PSProvider FileSystem -Root D:%5c%5cOneDrive%5c%5cDocuments%5c%5cGitHub\u000A    %7d\u000A    Set-Location \u0027Git:%5c%5cPresentations%5c%5c2019%5c%5cSQL Saturday Oslo\u0027\u000A%7d\u000A\u000AGet-ChildItem\u000A \u000D %22%7D">Set Location and List FIles</a>

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 Import-Module GIT:%5c%5cTechorama-Precon-2019%5c%5cSlides-and-Demos%5c%5c07-Programming%5c%5czNothing.psm1 -DisableNameChecking\u000A\u000ANothing-ElseMatters\u000AEnter-Sandman\u000AFor-WhomTheBellTolls \u000D %22%7D">Play Some Metallica</a>